In [1]:
import matplotlib as mpl
mpl.use("pgf")
# from palettable.tableau import Tableau_10
# from cycler import cycler
import numpy as np
pgf_with_pdflatex = {
    "pgf.texsystem": "pdflatex",
    "pgf.preamble": [
         r"\usepackage[utf8x]{inputenc}",
         r"\usepackage[T1]{fontenc}",
         r"\usepackage{mathptmx}",
         r"\usepackage{mathtools}",
#          r"\usepackage{biblatex}",
         ],
    "text.usetex": True,
    "figure.figsize": [4, 3.5],
    "axes.labelsize": 12,
    "axes.labelpad": 5.0,
    "font.size": 12,
    "font.family": "serif",
    "legend.fontsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "lines.linewidth": 1.5,
#     "axes.prop_cycle": cycler(color=Tableau_10.mpl_colors),
    "xtick.major.size": 5,
    "xtick.major.width": 1,
    "xtick.minor.size": 2.5,
    "xtick.minor.width": 1,
    "ytick.major.size": 5,
    "ytick.major.width": 1,
    "ytick.minor.size": 2.5,
    "ytick.minor.width": 1,
}
mpl.rcParams.update(pgf_with_pdflatex)

import uconnrcmpy as ucr
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, AutoMinorLocator

In [2]:
reac = ucr.experiments.Experiment(Path('ignition-delay-definition/20_in_00_mm_338K-865t-100x-24-Jan-17-1545.txt'), cti_file='./species.xml')
rpt = reac.pressure_trace
rpt.zeroed_time *= 1000
# reac.ignition_delay += 0.5
nonr = ucr.experiments.Experiment(Path('ignition-delay-definition/NR_20_in_00_mm_338K-865t-100x-16-Feb-17-1528.txt'), cti_file='./species.xml')
npt = nonr.pressure_trace
npt.zeroed_time *= 1000
# npt.zeroed_time += 0.5



**** WARNING ****
For species oh*, discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  53.6206
	Value computed using high-temperature polynomial: 53.5842


**** WARNING ****
For species ch3o2, discontinuity in cp/R detected at Tmid = 1000
	Value computed using low-temperature polynomial:  11.5313
	Value computed using high-temperature polynomial: 11.5482


**** WARNING ****
For species ch*, discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  107.505
	Value computed using high-temperature polynomial: 107.348


**** WARNING ****
For species c2h2oh, discontinuity in cp/R detected at Tmid = 1000
	Value computed using low-temperature polynomial:  12.2546
	Value computed using high-temperature polynomial: 12.2718


**** WARNING ****
For species ohv, discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  53.6206
	Value computed using high-temperature polynomial

In [3]:
fig, pressure_ax = plt.subplots()
pressure_ax.set_xlabel('Time, ms')
pressure_ax.set_ylabel('Pressure, bar')
pressure_ax.xaxis.set_major_locator(MaxNLocator(6))
pressure_ax.xaxis.set_minor_locator(AutoMinorLocator(4))
pressure_ax.yaxis.set_minor_locator(AutoMinorLocator(2))
pressure_ax.plot(rpt.zeroed_time, rpt.pressure, label='Pressure')
pressure_ax.plot(npt.zeroed_time, npt.pressure, label='Non-Reactive\nPressure')

dpdt_ax = pressure_ax.twinx()
dpdt_max = np.argmax(rpt.derivative)
dpdt_ax.set_ylabel('Time Derivative of Pressure, bar/ms')
dpdt_ax.plot(rpt.zeroed_time[:dpdt_max], rpt.derivative[:dpdt_max]/1000, label='Time Derivative',
             color=next(pressure_ax._get_lines.prop_cycler)['color'])
dpdt_ax.yaxis.set_minor_locator(AutoMinorLocator(2))
dpdt_ax.set_ylim(-0.25, 2.0)

pressure_ax.axvline(x=0, color='k', ymax=0.4)
pressure_ax.axvline(x=reac.ignition_delay, color='k')
pressure_ax.annotate(r'$\tau$', xy=(reac.ignition_delay/2, 20), xycoords='data', xytext=(0, 10), textcoords='offset points')
pressure_ax.annotate('', xy=(0, 20), xycoords='data', xytext=(reac.ignition_delay, 20), textcoords='data',
                     arrowprops={'arrowstyle': '<|-|>', 'shrinkA': 0, 'shrinkB': 0, 'fc': 'black'})
pressure_ax.annotate('EOC', xy=(0, 25), xycoords='data')
pressure_ax.annotate('', xy=(-6, 10), xycoords='data', xytext=(-12, 10), textcoords='data',
                     arrowprops={'arrowstyle': '<|-', 'shrinkA': 0, 'shrinkB': 0, 'fc': 'black'})
dpdt_ax.annotate('', xy=(-1, 1.0), xycoords='data', xytext=(12, 1.0), textcoords='data',
                 arrowprops={'arrowstyle': '<|-', 'shrinkA': 0, 'shrinkB': 0, 'fc': 'black'})
pressure_ax.set_ylim(0, 60)
pressure_ax.set_xlim(-15, 40)
pressure_ax.set_xlabel('Time, ms')
pressure_ax.set_ylabel('Pressure, bar')

lin_1, leg_1 = pressure_ax.get_legend_handles_labels()
lin_2, leg_2 = dpdt_ax.get_legend_handles_labels()
dpdt_ax_legend = dpdt_ax.legend(lin_1 + lin_2, leg_1 + leg_2, bbox_to_anchor=(0.3, 0.95))

for leg in dpdt_ax_legend.legendHandles:
    leg.set_linewidth(2.0)

In [4]:
fig.savefig('ignition_delay_definition.pgf', bbox_inches='tight')